In [1]:
from resume import my_resume

In [2]:
from IPython.display import display, HTML, Markdown, clear_output, FileLink
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights
import os, shutil, re
from mark_status import mark_status
import requests

In [3]:
# my_resume.write_html_to_file(
#     stylesheet='chalkboard',
#     filepath="docs/index.html",
#     should_render_all=True
# )

In [4]:
# my_resume.write_html_to_file(
#     stylesheet='lighttheme',
#     filepath="docs/resumes/index.html",
#     should_render_all=True
# )

In [5]:
# my_resume.write_html_to_file(
#     stylesheet='lighttheme',
#     filepath="docs/resumes/full.html",
#     should_render_all=True,
#     alt_template_prefixes={'*':'pdf'},
# )

In [6]:
# my_resume.export_pdf(
#     "docs/resumes/full.pdf",
#     # html_docs_subpath="resumes/full.html",

#     should_render_all=True,

#     stylesheet='lighttheme',
#     alt_template_prefixes={'*':'pdf'}
# )

# Pulling Job Data

In [7]:
jobs = get_jobs().query("company_name=='Sandia National Laboratories'")
raw_job_details = get_raw_job_details(jobs)

In [8]:
job_descriptions = skill_weights.get_job_descriptions(raw_job_details)
job_skill_weights = skill_weights.get_job_skill_weights(raw_job_details)

# Generation

In [9]:
skill_text_weights_con = lambda weights, job_id: weights.loc[job_id, :].groupby(level=0).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [10]:
os.makedirs("docs/resumes/samples/sandia/v1/", exist_ok=True)

In [11]:
get_jobid = re.compile(r"(?<=Job ID: )\d+")
jobs['Sandia job ID'] = job_descriptions["job description"].apply(lambda desc: get_jobid.search(desc)[0])

In [12]:
jobs["Sandia job ID"]

id
8953f570-c924-46e2-8e89-ab4c050cb05d    686026
1d871c9a-89c2-415c-bb05-caf7cd7dad70    685688
4c73950e-c311-4e91-acec-f9a8df9dc71f    685916
900d0476-89ca-421c-9d90-4bf8e42ac878    685883
4e222212-339d-442d-ae42-c3fd0cc2adf3    685897
Name: Sandia job ID, dtype: object

In [13]:
jobs['Sandia job ID'].apply(lambda sandia_job_id_str: requests.get(
    f"https://sandia.jobs/talent/search?q={sandia_job_id_str}", 
    headers={"Content-type":'application/json', "Accept":'application/json'}
).json()['jobs'])

id
8953f570-c924-46e2-8e89-ab4c050cb05d    [{'job': {'name': 'projects/cloudjobsearch-256...
1d871c9a-89c2-415c-bb05-caf7cd7dad70    [{'job': {'name': 'projects/cloudjobsearch-256...
4c73950e-c311-4e91-acec-f9a8df9dc71f    [{'job': {'name': 'projects/cloudjobsearch-256...
900d0476-89ca-421c-9d90-4bf8e42ac878    [{'job': {'name': 'projects/cloudjobsearch-256...
4e222212-339d-442d-ae42-c3fd0cc2adf3    [{'job': {'name': 'projects/cloudjobsearch-256...
Name: Sandia job ID, dtype: object

In [14]:
jobs['Sandia url'] = jobs['Sandia job ID'].apply(lambda sandia_job_id_str: "https://sandia.jobs"+requests.get(
    f"https://sandia.jobs/talent/search?q={sandia_job_id_str}", 
    headers={"Content-type":'application/json', "Accept":'application/json'}
).json()['jobs'][0]['job']['detailLink'])

In [15]:
jobs['comparison'] = jobs['Sandia job ID'].apply("<a href='/resumes/samples/sandia/v1/{0}_compare.html'>Compare</a>".format)

In [16]:
jobs['note'] = jobs.apply(lambda row: raw_job_details[row.name]["attributes"]['note'], axis='columns')

In [17]:
for job_id, job_data in jobs.iterrows():
    sandia_id = job_data['Sandia job ID']
    pdf_url = f"/resumes/samples/sandia/v1/{sandia_id}_Zach_Allen_Resume.pdf"
    my_resume.export_fitted_pdf(
        "docs"+pdf_url,

        highest_threshold=0.05,
        lowest_threshold=-1e-3,
        skill_cost=0.1,
        should_render_all_skills=True,

        stylesheet='lighttheme',
        alt_template_prefixes={'*':'pdf'},
        skill_text_weights = skill_text_weights_con(job_skill_weights, job_id)
    )
    
    with open(f"docs/resumes/samples/sandia/v1/{sandia_id}_compare.html", 'w+') as compare_f:
        compare_f.write(f"<div style='display:flex; width:100%; height:100%;'><iframe src='{job_data['Sandia url']}' style='flex:auto'></iframe><iframe src='{pdf_url}#toolbar=0' style='aspect-ratio: 8.5 / 11;'></iframe></div>")


Must traverse configuration space.
Must traverse configuration space.
Must traverse configuration space.
Must traverse configuration space.
Must traverse configuration space.


In [18]:
# assert input("Ready to generate? Type 'OK'") == "OK"
with open(f"docs/resumes/samples/sandia/v1/index.html", 'w+') as sandia_index:
    sandia_index.write("<h1>Customized Resumes for Sandia Postings</h1><p>Below are links to resumes which have been paired with and customized for selected job postings at Sandia National Laboratories.</p>")
    jobs.set_index("Sandia job ID")[["location", "role", 'note', "comparison"]].sort_index().to_html(buf=sandia_index, escape=False)

In [19]:
assert False

AssertionError: 

In [ ]:
pdf_upload_path = "docs/pdf_sources/upload/Zach_Allen_Resume.pdf"
os.makedirs("docs/pdf_sources/upload", exist_ok=True)

In [ ]:
# global_skill_text_weights = job_skill_weights.groupby(level=["id", "skill", "skill text"]
#         ).sum().groupby(level=["skill", "skill text"]
#         ).sum().divide(len(jobs)).groupby(level=0
#         ).apply(lambda df: df.xs(df.name).to_dict('index')).to_dict()

In [ ]:
for job_id, job_data in jobs.sort_index().iterrows():
    clear_output()
    
    # Display links to job info
    print(job_id)
    print(job_data['company_name'])
    display(HTML(f"<a href='{job_data['url']}'>job</a>"))
    
    # Construct the PDF
    os.makedirs(f"docs/pdf_sources/{job_id}", exist_ok=True)
    pdf_fpath = f"docs/pdf_sources/{job_id}/Zach_Allen_Resume.pdf"
    my_resume.export_fitted_pdf(
        pdf_fpath,
        
        highest_threshold=0.05,
        lowest_threshold=-1e-3,
        skill_cost=0.1,
        should_render_all_skills=True,
        
        stylesheet='lighttheme',
        alt_template_prefixes={'*':'pdf'},
        skill_text_weights = skill_text_weights_con(job_skill_weights, job_id)
    )
    
    # Copy the output PDF to a single location for easy upload
    try:
        shutil.copyfile(pdf_fpath, pdf_upload_path);
    except shutil.SameFileError: # Sometimes this comes up when re-running a job
        pass
    
    # Wait for feedback on submission status
    status = input("Status:")
    if status == "skip":
        continue;
    if status == "":
        status = "applied"
    
    mark_status(job_id, status_key_or_id = status)
    